In [37]:
import pymysql
import pymysql.cursors

connection = pymysql.connect(host='mysql',
                             user='PHYS7332-user',
                             database='PHYS7332',
                             password='7332.202510',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [38]:
with connection:
    with connection.cursor() as cursor:
        sql = """
        CREATE TABLE `fla_species` (
            `node_id` int(11) NOT NULL,
            `name` varchar(255)  COLLATE utf8mb4_bin NOT NULL,
            `group` varchar(255) COLLATE utf8mb4_bin NULL,
        PRIMARY KEY (`node_id`))
        ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_bin;
        """
        cursor.execute(sql)

OperationalError: (1142, "CREATE command denied to user 'PHYS7332-user'@'c0744' for table 'fla_species'")